In [40]:
import numpy as np
import pandas as pd
import datetime as dt

In [41]:
prices = pd.read_csv('prices.csv')
sales = pd.read_csv('sales.csv')

In [42]:
prices['updated_at'] = pd.to_datetime(prices['updated_at'])
sales['ordered_at'] = pd.to_datetime(pd.to_datetime(sales['ordered_at']).dt.strftime('%Y-%m-%d %H:%M:%S'))

In [43]:
prices.info()
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   product_id  14 non-null     int64         
 1   old_price   14 non-null     int64         
 2   new_price   14 non-null     int64         
 3   updated_at  14 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(3)
memory usage: 576.0 bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   product_id        175 non-null    int64         
 1   ordered_at        175 non-null    datetime64[ns]
 2   quantity_ordered  175 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.2 KB


In [44]:
def returned(id, time):
    max_update = prices[(id == prices.product_id) & (time > prices.updated_at )].updated_at.max()

    get_price = prices[(prices.product_id == id) & (prices.updated_at == max_update)].new_price

    if get_price.empty == True:
        min_update = prices[(id == prices.product_id)].updated_at.min()
        get_price = prices[(prices.product_id == id) & (prices.updated_at == min_update)].old_price
    
    return(get_price)

In [45]:
sales['price'] = pd.Series(dtype='int')

In [46]:
for i in sales.index:
         sales.price[i] = returned(sales.product_id[i], sales.ordered_at[i])

<ipython-input-46-8ac9c98de8f5>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales.price[i] = returned(sales.product_id[i], sales.ordered_at[i])


In [47]:
sales['revenue'] = sales.quantity_ordered * sales.price

In [48]:
sales.head(20)

,product_id,ordered_at,quantity_ordered,price,revenue
0,3998909,2018-09-18 17:51:00,1,16500.0,16500.0
1,3998909,2018-09-18 12:52:00,1,16500.0,16500.0
2,3998909,2018-09-18 11:33:00,1,16500.0,16500.0
3,3998909,2018-09-18 18:47:00,1,16500.0,16500.0
4,3998909,2018-09-18 17:36:00,1,16500.0,16500.0
5,3998909,2018-09-17 19:09:00,1,16500.0,16500.0
6,3998909,2018-09-18 16:02:00,1,16500.0,16500.0
7,3998909,2018-09-17 22:29:00,1,16500.0,16500.0
8,3998909,2018-09-18 19:46:00,2,16500.0,33000.0
9,3998909,2018-09-18 11:02:00,1,16500.0,16500.0


In [49]:
sales.groupby(['product_id', 'price'])['revenue'].sum()

product_id  price   
64          239000.0     956000.0
3954203     57500.0       57500.0
            60000.0      180000.0
            64000.0      640000.0
3998909     15500.0       15500.0
            16500.0      231000.0
            17000.0       34000.0
4085861     52000.0     1040000.0
            53500.0     2140000.0
            58000.0     2204000.0
            60000.0      180000.0
            62500.0     1812500.0
            67000.0      871000.0
Name: revenue, dtype: float64

In [50]:
sales.pivot_table(
    values= 'revenue',
    index= ['product_id', 'price'],
    aggfunc= sum
)

revenue
product_id price              
64         239000.0   956000.0
3954203    57500.0     57500.0
           60000.0    180000.0
           64000.0    640000.0
3998909    15500.0     15500.0
           16500.0    231000.0
           17000.0     34000.0
4085861    52000.0   1040000.0
           53500.0   2140000.0
           58000.0   2204000.0
           60000.0    180000.0
           62500.0   1812500.0
           67000.0    871000.0